In [1]:
import torch
import syft as sy
from torch import nn, optim
import torch.nn.functional as f

In [2]:
class network(nn.Module):
    def __init__(self):
        super(network, self).__init__()
        self.fc1 = nn.Linear(2,2)
        self.fc2 = nn.Linear(2,1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = f.relu(x)
        x = self.fc2(x)
        return x

model = network()

In [3]:
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
trusted_crypto_provider = sy.VirtualWorker(hook, id="trusted_crypto_provider")

In [4]:
x = torch.tensor([[1.,2],[2,3],[4,5],[2,4]])
y = torch.tensor([[1.],[0],[0],[1]])

In [5]:
x = x.fix_precision().share(bob, alice, crypto_provider=trusted_crypto_provider, requires_grad=True)
y = y.fix_precision().share(bob, alice, crypto_provider=trusted_crypto_provider, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=trusted_crypto_provider, requires_grad=True)

optimizer = optim.SGD(model.parameters(), lr=0.003).fix_precision()

In [6]:
for epoch in range(50):
    for i in range(10):
        encrypted_pred = model(x)
        optimizer.zero_grad()
        loss = ((encrypted_pred - y)**2).sum()
        loss.backward()
        optimizer.step()
    #print loss after each epoch    
    print(" Loss: " + str(loss.get().float_precision()))       

 Loss: tensor([2.3680])
 Loss: tensor([2.1260])
 Loss: tensor([1.9220])
 Loss: tensor([1.7360])
 Loss: tensor([1.5740])
 Loss: tensor([1.4250])
 Loss: tensor([1.2940])
 Loss: tensor([1.1790])
 Loss: tensor([1.0650])
 Loss: tensor([0.9660])
 Loss: tensor([0.8720])
 Loss: tensor([0.7910])
 Loss: tensor([0.7210])
 Loss: tensor([0.6580])
 Loss: tensor([0.6020])
 Loss: tensor([0.5540])
 Loss: tensor([0.5140])
 Loss: tensor([0.4790])
 Loss: tensor([0.4480])
 Loss: tensor([0.4240])
 Loss: tensor([0.4020])
 Loss: tensor([0.3840])
 Loss: tensor([0.3690])
 Loss: tensor([0.3560])
 Loss: tensor([0.3480])
 Loss: tensor([0.3380])
 Loss: tensor([0.3340])
 Loss: tensor([0.3260])
 Loss: tensor([0.3250])
 Loss: tensor([0.3210])
 Loss: tensor([0.3180])
 Loss: tensor([0.3190])
 Loss: tensor([0.3160])
 Loss: tensor([0.3130])
 Loss: tensor([0.3110])
 Loss: tensor([0.3100])
 Loss: tensor([0.3080])
 Loss: tensor([0.3080])
 Loss: tensor([0.3060])
 Loss: tensor([0.3060])
 Loss: tensor([0.3030])
 Loss: tensor([0

In [7]:
model.get().float_precision()
print(model.fc1.weight.data)
print(model.fc1.bias.data)
print(model.fc2.weight.data)
print(model.fc2.bias.data)

tensor([[0.8490, 0.1300],
        [0.1000, 0.9760]])
tensor([-0.7090, -0.2260])
tensor([[-0.9420,  0.5830]])
tensor([0.0770])
